In [106]:
import numpy as np
from ase.io import read, write
import os

In [107]:
def get_cube_data(cube_file):
    with open(cube_file, 'r') as f:
            cube_data = f.readlines()

    # Number of atoms, origin, FFT grid size and voxel dimensions
    natoms = int(cube_data[2].split()[0])
    origin = np.array([cube_data[2].split()[1], cube_data[2].split()[2], cube_data[2].split()[3]], dtype='float64')
    N = np.array([cube_data[3].split()[0], cube_data[4].split()[0], cube_data[5].split()[0]], dtype='int64')
    dV = np.array([[cube_data[3].split()[1], cube_data[3].split()[2], cube_data[3].split()[3]],
                [cube_data[4].split()[1], cube_data[4].split()[2], cube_data[4].split()[3]],
                [cube_data[5].split()[1], cube_data[5].split()[2], cube_data[5].split()[3]]], dtype='float64')

    # Convert from Bohr to Angstroms (uncomment if required)
    # dV *= 0.529177249

    # Volume of a grid
    dvolume =  np.dot(dV[0], np.cross(dV[1], dV[2]))

    # Read density data as a numpy array
    density_data = np.loadtxt(cube_file, skiprows=6+natoms)

    # The way charges are stored in a cube file with innermost loop on z and outermost on x
    density_data = density_data.reshape((N[0], N[1], N[2]), order='C')
    
    # Charge contained in a grid
    charge_data = density_data*dvolume
    
    return charge_data
    
def write_CHGCAR(structure_file, cube_file, chgcar_file):
    
    write('temp_POSCAR', read(structure_file), format='vasp')
    
    charge_data = get_cube_data(cube_file)

    Nx, Ny, Nz = charge_data.shape[0], charge_data.shape[1], charge_data.shape[2]
    
    with open('temp_POSCAR', 'r') as f:
        poscar_lines = f.readlines()
    
    with open(chgcar_file, 'w') as f:
        for i in range(len(poscar_lines)):
            f.write(poscar_lines[i])
            
        f.write('\n')
        
        f.write(f'   {Nx}   {Ny}   {Nz}')
        f.write('\n')
        count = 0

        # The way charges are stored in CHGCAR, innermost loop on x, outermost on z
        for k in range(Nz):
            for j in range(Ny):
                for i in range(Nx):
                    f.write(f' {charge_data[i, j, k]*Nx*Ny*Nz:.12f}')
                    count += 1
                    if count % 5 == 0:
                        f.write('\n')
    
        f.write('\n')
    os.system("rm temp_POSCAR")
    return